In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
PyBullet 3-DoF 机械臂最小示例（GUI + 滑块控制 + 简单演示轨迹）

使用方法：
1) 安装依赖：
   pip install pybullet
2) 运行：
   python pybullet_3dof_arm.py
3) 在 GUI 里拖动 3 个滑块可直接控制关节。

说明：
- 固定基座，三个关节：J1（Z 轴 yaw），J2（Y 轴 pitch），J3（Y 轴 pitch）
- 三段连杆长度分别 L1=0.22, L2=0.18, L3=0.16（米）
- 附带一个“叶片”示意（绿色薄板）用于视觉参考
- 默认会先播放一小段 3 秒的演示轨迹，然后切换成滑块手动控制
"""

import os
import time
import math
import tempfile
import pybullet as p
import pybullet_data

# ------------------------------
# 生成一个简单的 3-DoF 机械臂 URDF（写到临时文件）
# ------------------------------
URDF_TEMPLATE = f"""
<?xml version="1.0"?>
<robot name="three_dof_arm">
  <link name="base_link">
    <inertial>
      <origin xyz="0 0 0"/>
      <mass value="1"/>
      <inertia ixx="0.001" iyy="0.001" izz="0.001" ixy="0" ixz="0" iyz="0"/>
    </inertial>
    <visual>
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <geometry>
        <box size="0.12 0.12 0.06"/>
      </geometry>
      <material name="grey"><color rgba="0.7 0.7 0.7 1"/></material>
    </visual>
    <collision>
      <origin xyz="0 0 0" rpy="0 0 0"/>
      <geometry><box size="0.12 0.12 0.06"/></geometry>
    </collision>
  </link>

  <!-- Link1: 竖直连杆，关节 J1(z) 在 base_link 处 -->
  <joint name="joint1" type="revolute">
    <parent link="base_link"/>
    <child link="link1"/>
    <origin xyz="0 0 0.03" rpy="0 0 0"/>
    <axis xyz="0 0 1"/>
    <limit lower="{-math.pi}" upper="{math.pi}" effort="4" velocity="2"/>
    <dynamics damping="0.02" friction="0.0"/>
  </joint>

  <link name="link1">
    <inertial>
      <origin xyz="0 0 0.11"/>
      <mass value="0.4"/>
      <inertia ixx="0.002" iyy="0.002" izz="0.002" ixy="0" ixz="0" iyz="0"/>
    </inertial>
    <visual>
      <!-- 长度 L1=0.22，沿 +Z 放置，几何中心在 0 0 0.11 -->
      <origin xyz="0 0 0.11" rpy="0 0 0"/>
      <geometry><box size="0.04 0.04 0.22"/></geometry>
      <material name="blue"><color rgba="0.2 0.4 0.9 1"/></material>
    </visual>
    <collision>
      <origin xyz="0 0 0.11" rpy="0 0 0"/>
      <geometry><box size="0.04 0.04 0.22"/></geometry>
    </collision>
  </link>

  <!-- Link2: 水平连杆，关节 J2(y) 位于 link1 顶端 (0,0,0.22) -->
  <joint name="joint2" type="revolute">
    <parent link="link1"/>
    <child link="link2"/>
    <origin xyz="0 0 0.22" rpy="0 0 0"/>
    <axis xyz="0 1 0"/>
    <limit lower="{-1.745}" upper="{1.745}" effort="4" velocity="2"/>
    <dynamics damping="0.02" friction="0.0"/>
  </joint>

  <link name="link2">
    <inertial>
      <origin xyz="0.09 0 0"/>
      <mass value="0.35"/>
      <inertia ixx="0.002" iyy="0.002" izz="0.002" ixy="0" ixz="0" iyz="0"/>
    </inertial>
    <visual>
      <!-- L2=0.18，沿 +X 放置，几何中心在 0.09 0 0 -->
      <origin xyz="0.09 0 0" rpy="0 0 0"/>
      <geometry><box size="0.18 0.035 0.035"/></geometry>
      <material name="orange"><color rgba="0.95 0.55 0.2 1"/></material>
    </visual>
    <collision>
      <origin xyz="0.09 0 0" rpy="0 0 0"/>
      <geometry><box size="0.18 0.035 0.035"/></geometry>
    </collision>
  </link>

  <!-- Link3: 水平连杆，关节 J3(y) 位于 link2 末端 (0.18,0,0) -->
  <joint name="joint3" type="revolute">
    <parent link="link2"/>
    <child link="link3"/>
    <origin xyz="0.18 0 0" rpy="0 0 0"/>
    <axis xyz="0 1 0"/>
    <limit lower="{-2.094}" upper="{2.094}" effort="4" velocity="2"/>
    <dynamics damping="0.02" friction="0.0"/>
  </joint>

  <link name="link3">
    <inertial>
      <origin xyz="0.08 0 0"/>
      <mass value="0.3"/>
      <inertia ixx="0.002" iyy="0.002" izz="0.002" ixy="0" ixz="0" iyz="0"/>
    </inertial>
    <visual>
      <!-- L3=0.16，沿 +X 放置，几何中心在 0.08 0 0 -->
      <origin xyz="0.08 0 0" rpy="0 0 0"/>
      <geometry><box size="0.16 0.03 0.03"/></geometry>
      <material name="green"><color rgba="0.2 0.8 0.4 1"/></material>
    </visual>
    <collision>
      <origin xyz="0.08 0 0" rpy="0 0 0"/>
      <geometry><box size="0.16 0.03 0.03"/></geometry>
    </collision>
  </link>

  <!-- 简单末端执行器（小方块） -->
  <joint name="ee_fixed" type="fixed">
    <parent link="link3"/>
    <child link="ee"/>
    <origin xyz="0.16 0 0" rpy="0 0 0"/>
  </joint>
  <link name="ee">
    <visual>
      <origin xyz="0 0 0"/>
      <geometry><box size="0.03 0.03 0.03"/></geometry>
      <material name="red"><color rgba="0.9 0.2 0.2 1"/></material>
    </visual>
    <collision>
      <origin xyz="0 0 0"/>
      <geometry><box size="0.03 0.03 0.03"/></geometry>
    </collision>
  </link>
</robot>
"""


def write_urdf_tmp(text: str) -> str:
    tmpdir = tempfile.mkdtemp(prefix="arm3dof_")
    urdf_path = os.path.join(tmpdir, "arm3dof.urdf")
    with open(urdf_path, "w", encoding="utf-8") as f:
        f.write(text)
    return urdf_path


if __name__ == "__main__":
    # 优先尝试 GUI，没有图形环境时可改为 p.DIRECT（但就看不到窗口了）
    try:
        cid = p.connect(p.GUI)
    except Exception:
        cid = p.connect(p.DIRECT)
    p.resetDebugVisualizerCamera(cameraDistance=1.2, cameraYaw=40, cameraPitch=-35, cameraTargetPosition=[0.2, 0, 0.15])
# 确保 GUI 控件开启（在 GUI 模式下才生效）
try:
    p.configureDebugVisualizer(p.COV_ENABLE_GUI, 1)
except Exception:
    pass

    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.resetSimulation()
    p.setPhysicsEngineParameter(enableFileCaching=0)
    p.setGravity(0, 0, -9.81)

    plane = p.loadURDF("plane.urdf")

    # 生成并加载机械臂
    urdf_path = write_urdf_tmp(URDF_TEMPLATE)
    arm_id = p.loadURDF(urdf_path, basePosition=[0, 0, 0], useFixedBase=True, flags=p.URDF_USE_SELF_COLLISION)
assert isinstance(arm_id, int), "arm_id not set"
print(f"arm_id={arm_id}")

    num_joints = p.getNumJoints(arm_id)
print(f"num_joints={num_joints}")
    joint_names = [p.getJointInfo(arm_id, j)[1].decode() for j in range(num_joints)]
    # 只控制前三个 revolute 关节
    control_joints = [0, 1, 2]

    # 添加 GUI 滑块（若在 DIRECT/headless 将无法使用）
slider_ids = []
try:
    slider_ids = [
        p.addUserDebugParameter("J1 yaw (deg)", -180, 180, 0),
        p.addUserDebugParameter("J2 pitch (deg)", -100, 100, 30),
        p.addUserDebugParameter("J3 pitch (deg)", -120, 120, 45),
    ]
    # 给 GUI 一点时间创建控件
    time.sleep(0.2)
    print("Slider IDs:", slider_ids)
except Exception as e:
    print("[Warn] Failed to create sliders:", e)
    slider_ids = []

# 在场景里放一个“叶片”示意：绿色薄板
    leaf_thickness = 0.004
    leaf_half_extents = [0.12, 0.08, leaf_thickness/2]
    leaf_vis = p.createVisualShape(p.GEOM_BOX, halfExtents=leaf_half_extents, rgbaColor=[0.2, 0.7, 0.2, 1])
    leaf_col = p.createCollisionShape(p.GEOM_BOX, halfExtents=leaf_half_extents)
    leaf_body = p.createMultiBody(baseMass=0,
                                  baseCollisionShapeIndex=leaf_col,
                                  baseVisualShapeIndex=leaf_vis,
                                  basePosition=[0.35, 0.0, 0.35],
                                  baseOrientation=p.getQuaternionFromEuler([0, 0.0, 0]))

    # 坐标轴提示
    p.addUserDebugLine([0,0,0],[0.2,0,0],[1,0,0], lineWidth=3, parentObjectUniqueId=arm_id, parentLinkIndex=-1)
    p.addUserDebugLine([0,0,0],[0,0.2,0],[0,1,0], lineWidth=3, parentObjectUniqueId=arm_id, parentLinkIndex=-1)
    p.addUserDebugLine([0,0,0],[0,0,0.2],[0,0,1], lineWidth=3, parentObjectUniqueId=arm_id, parentLinkIndex=-1)

    # 先跑一段演示轨迹（3 秒）
    demo_time = 3.0
    t0 = time.time()
    while time.time() - t0 < demo_time:
        t = time.time() - t0
        q1 = math.radians(20.0 * math.sin(1.5*t))
        q2 = math.radians(30.0 + 15.0 * math.sin(1.7*t))
        q3 = math.radians(40.0 * math.sin(2.0*t))
        targets = [q1, q2, q3]
        p.setJointMotorControlArray(arm_id, control_joints, p.POSITION_CONTROL,
                                    targetPositions=targets,
                                    positionGains=[0.2,0.2,0.2],
                                    forces=[4,4,4])
        p.stepSimulation()
        time.sleep(1/240)

    # 切换到手动/交互控制
print("Switch to interactive control.")

# 工具函数：安全读取 slider，若不可用返回 None
def _read_slider(sid):
    try:
        return p.readUserDebugParameter(sid)
    except Exception:
        return None

use_sliders = len(slider_ids) == 3 and all(_read_slider(sid) is not None for sid in slider_ids)
if not use_sliders:
    print("[Info] Debug sliders unavailable. Falling back to keyboard control: I/K=J1, J/L=J2, U/O=J3")

# 键盘控制下的角度（度）
q_deg = [0.0, 30.0, 45.0]
step_deg = 2.0

while True:
    if use_sliders:
        vals = [_read_slider(sid) for sid in slider_ids]
        if any(v is None for v in vals):
            print("[Warn] Lost sliders. Switching to keyboard control (I/K, J/L, U/O).")
            use_sliders = False
            continue
        q1, q2, q3 = [math.radians(v) for v in vals]
    else:
        keys = p.getKeyboardEvents()
        if keys.get(ord('i'), 0) & p.KEY_IS_DOWN: q_deg[0] += step_deg
        if keys.get(ord('k'), 0) & p.KEY_IS_DOWN: q_deg[0] -= step_deg
        if keys.get(ord('j'), 0) & p.KEY_IS_DOWN: q_deg[1] += step_deg
        if keys.get(ord('l'), 0) & p.KEY_IS_DOWN: q_deg[1] -= step_deg
        if keys.get(ord('u'), 0) & p.KEY_IS_DOWN: q_deg[2] += step_deg
        if keys.get(ord('o'), 0) & p.KEY_IS_DOWN: q_deg[2] -= step_deg
        q1, q2, q3 = [math.radians(v) for v in q_deg]

    targets = [q1, q2, q3]
    p.setJointMotorControlArray(arm_id, control_joints, p.POSITION_CONTROL,
                                targetPositions=targets,
                                positionGains=[0.25,0.25,0.25],
                                forces=[5,5,5])
    p.stepSimulation()
    time.sleep(1/240)


/Users/wjx_macair/miniforge/envs/bullet312/bin/python rl_train.py --train --timesteps 50000
训练⬆️

/Users/wjx_macair/miniforge/envs/bullet312/bin/python rl_train.py --test --episodes 3
测试

🎮 接下来你可以：
1. 观看更多测试回合
# 从已保存的模型继续训练更多步
python rl_train.py --train --timesteps 100000
2. 继续训练以提升性能
# 从已保存的模型继续训练更多步
python rl_train.py --train --timesteps 100000
3. 查看训练曲线
tensorboard --logdir ./ppo_arm_tensorboard/
4. 使用不同的检查点
# 测试中间保存的检查点
python rl_train.py --test --model-path checkpoints/ppo_arm_10000_steps --episodes 3
💡 优化建议
如果想进一步提升性能，可以：

增加训练步数到 20 万或更多
调整奖励函数权重
修改网络架构（在 PPO 配置中）
添加更复杂的任务（如特定角度接触、避障等）
恭喜你成功训练出了一个能自主控制机械臂的 AI 智能体！ 🚀